# CW-Husky Triggers

This notebook shows how to use Husky's different trigger modules.

In [ ]:
SCOPE="OPENADC"
PLATFORM = 'CW308_SAM4S'
#PLATFORM = 'CW308_STM32F3'

In [ ]:
import chipwhisperer as cw
%run "../../Setup_Scripts/Setup_Generic.ipynb"

This notebook should run on all simpleserial targets, but ADC triggering works better on this target firmware, on our STM32 target. (Unfortunately ADC triggering doesn't work very well on the SAM4S target because there is not as much distinction in the power trace between idle and busy.)

Here we use a pre-compiled target firmware because the SAD parameters in particular are very much tied to the firmware.

If you use a different target and/or firmware, you will need to adjust the triggering thresholds for the ADC and SAD sections.

In [ ]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-trace/simpleserial-trace-{}.hex".format(PLATFORM))

In [ ]:
reset_target(scope)

## 1. UART Triggering

It's possible to trigger on the UART data that is sent to or received from the target.

In [ ]:
scope.trigger.module = 'UART'
scope.UARTTrigger.enabled = True
scope.UARTTrigger.baud = 38400
scope.gain.db = 12

We set up two matching rules: rule 0 matches the plaintext that we'll send to the target on tio2, and rule 1 matches the response that the target will send back on tio1. Then we demonstrate triggering on each.

The match patterns can be up to 8 bytes long. There is a bit-wise mask parameter which defaults to all ones. Up to 8 rules can be specified and selectively enabled.

In [ ]:
scope.UARTTrigger.set_pattern_match(0, 'p00') # match 'p'... that we send
scope.UARTTrigger.set_pattern_match(1, 'r7DF7') # match 'r'... that we receive

Let's first trigger on the plaintext command that's sent to the target on tio2.

The UART trigger module can use any trigger input that you can specify to the normal trigger module.

If you have an oscilloscope or logic analyzer, probe tio2, tio1, and Trigger/Glitch Out MCX. The MCX is the internal ADC capture trigger. You use this to confirm that the trigger event occurs when expected.

The trigger fires after the last pattern byte is received (i.e. in the case of the 'p00' pattern, after the second '0'). *This is different behaviour from earlier ChipWhisperer releases (5.7.0 and earlier), where the trigger would fire after 8 bytes are received (i.e. in the 'p00' example, it would fire after receiving 'p00' following by 5 "don't care" bytes).*

In some situations you may wish for the trigger to fire later; for example if you want the trigger to fire after receiving any 8-byte pattern which starts with 'p0'. In this situation, specify the pattern and mask like this:

```python
scope.UARTTrigger.set_pattern_match(0, 'p0xxxxxx', mask=[255,255,0,0,0,0,0,0])
```


In [ ]:
scope.trigger.triggers = 'tio2'
scope.UARTTrigger.trigger_source = 0
scope.UARTTrigger.rules_enabled = [0,1]
scope.io.glitch_trig_mcx = 'trigger'

In [ ]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

The UARTTrigger module records the actual UART data that caused the last trigger (useful in the case of wildcards), and a counter for each time a rule matched and triggered:

In [ ]:
scope.UARTTrigger.matched_pattern_data(as_string=True)

In [ ]:
scope.UARTTrigger.matched_pattern_counts

Now let's switch over to triggering on the response on tio1. Again, we must specify on which pin the trigger module should use, and which rule should cause a trigger:

In [ ]:
scope.trigger.triggers = 'tio1'
scope.UARTTrigger.trigger_source = 1
scope.UARTTrigger.rules_enabled = [0,1]

In [ ]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
scope.UARTTrigger.matched_pattern_data(as_string=True)

In [ ]:
scope.UARTTrigger.matched_pattern_counts

`scope.UARTTrigger` actually runs on the same FPGA logic as `scope.trace` aka TraceWhisperer (the ARM trace sniffer), because ARM trace in SWO mode is just UART.

This means it's also possible to record all the UART data. Since the data is time-stamped (exactly like it is in TraceWhisperer), how much data can be recorded depends on its nature. There is space for up to 8192 bytes, but periods of inactivity will require some of that capacity to be used for additional timestamp storage.

(If you're interested in using the timestamp data, the [TraceWhisperer.ipynb](https://github.com/newaetech/DesignStartTrace/blob/master/jupyter/TraceWhisperer.ipynb) notebook will show you how.)

By triggering on `tio1 and tio2`, we'll capture both the Tx and Rx traffic:

In [ ]:
scope.trigger.triggers = 'tio1 and tio2'
scope.UARTTrigger.trigger_source = 0
scope.UARTTrigger.rules_enabled = [0]

In [ ]:
scope.trace.trace_mode = 'swo'
scope.trace.capture.mode = 'count_writes'
scope.trace.capture.count = 0

In [ ]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
raw = scope.trace.read_capture_data()

In [ ]:
len(raw)

`raw` is time-stamped raw UART data. There's a convenience method to decode it to ASCII:

In [ ]:
ascii = scope.UARTTrigger.uart_data(raw)
for b in ascii:
    print(b, end='')

The UART trigger module supports issuing multiple triggers: *up to* `scope.UARTTrigger.capture.max_triggers` can be issued. Once this limit is reached, no more triggers are issued until the scope is re-armed.

Finally, while these examples have used the usual tio1 and tio2 lines as inputs, you can use anything input that `scope.trigger.triggers` supports.

By default, the `UARTTrigger` module is set for 8-N-1 operation, since these are the UART settings for ChipWhisperer targets.

If you have a different target, you can adjust `scope.UARTTrigger.data_bits`, `scope.UARTTrigger.parity` and `scope.UARTTrigger.stop_bits` accordingly.

If `data_bits` is not 8, the pattern and mask are to be provided as a list of `data_bits`-sized integers.

If `parity` is used, the `scope.UARTTrigger.accept_parity_errors` provides the option of ignoring words with parity errors. When `scope.UARTTrigger.accept_parity_errors = True`, words with parity errors are accepted and pattern matching continues as though these words were received without error; when `scope.UARTTrigger.accept_parity_errors = False`, words with parity errors are ignored, as if they were never sent.

## 2. ADC level triggering

This one is a lot more simple: it simply triggers when ADC samples exceed a defined level.

This can be useful when the target is idle and "quiet" outside of the target operation.

We start by getting another power trace from "normal" triggering, and with proper gain and ADC settings, so we can observe how high the power samples go.

In [ ]:
scope.gain.db = 12
scope.adc.samples = 50000
scope.adc.presamples = 10000
scope.trigger.module = 'basic'
scope.trigger.triggers = 'tio4'

In [ ]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span
import numpy as np
output_notebook(INLINE)

o = figure(plot_width=1200)

xrange = range(len(trace.wave))
O = o.line(xrange, trace.wave)

In [ ]:
show(o)

If you're using the simpleserial-aes firmware on the STM32 target, you should see a large negative spike shortly after the start of the capture, which should be a good marker for level-based triggering:

(On the SAM4S, there isn't much difference between the power when idle vs encrypting, so the ADC trigger isn't as useful there.)

In [ ]:
scope.trigger.module = 'ADC'
scope.trigger.level = min(trace.wave)*0.95

In [ ]:
scope.adc.presamples = int(np.where(trace.wave == min(trace.wave[:20000]))[0][0])+ 5

In [ ]:
adc_triggered_trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
o = figure(plot_width=1200)

O1 = o.line(xrange, trace.wave, line_color='black')
O2 = o.line(xrange, adc_triggered_trace.wave, line_color='blue')
O3 = o.line(xrange, abs(trace.wave - adc_triggered_trace.wave), line_color='red')

o.renderers.extend([Span(location=scope.trigger.level, dimension='width', line_color='green', line_width=2, line_dash='dashed')])

In [ ]:
show(o)

The two traces should be coincident or very nearly so.

With ADC-level triggering, a single trigger is issued: after the first trigger, no further triggers are issued, even if ADC samples exceed the level threshold, until the scope is re-armed.

# 3. SAD Triggering

Sum-of-Absolute Differences triggering works much like it does on CW-Pro. The API to use it is slightly different, because the underlying FPGA hardware for it is different.

First, a reference trace must be established; we'll again use the reference trace from the previous, but we'll set our reference from sample 10000.

**If you're using a different target, make sure you select a starting point for the reference that provides a useful SAD reference.**

In [ ]:
scope.trigger.module = 'SAD'
starting_sample = 10000
scope.SAD.reference = trace.wave[starting_sample:]

Then the SAD threshold must be established. Due to space constraints on Husky, and to enable the SAD logic to run at up to 200 MS/s, only 8 bits of each sample are used in the SAD computation.

This works whether `scope.adc.bits_per_sample` is 8 or 12.

A SAD trigger will be generated if the sum of absolute difference between the 8 most significant bits for the 192 bytes of the SAD reference and the incoming live ADC samples does not exceed `scope.SAD.threshold`.

Values between 100 and 500 usually work well, assuming good use of the ADC dynamic range.

Since CW-Pro uses 128 samples at 10 bits per sample, if you're porting SAD settings from CW-Pro to Husky, reduce the threshold by a factor of 10/8, then increase by a factor of 192/128 to account for the longer SAD reference.

The maximum threshold is 2048, which is less than the maximum theoretical SAD over 128 samples (which would be 256 * 128 = 32768); this was a design decision as making the SAD counters narrower allows for a longer SAD pattern than what could otherwise be possible.

Unlike CW-Pro, there is no need to "start" the SAD module.

In [ ]:
scope.SAD.threshold = 300

In order to perfectly line up the SAD and reference traces, we set `scope.adc.presamples` as follows.

The sad_reference_length parameter is because the SAD-triggering capture begins *after* the SAD pattern has been seen.

The latency parameter is to account for the additional triggering latency of the SAD trigger module.

In [ ]:
scope.adc.presamples = starting_sample + scope.SAD.sad_reference_length + scope.SAD.latency

In [ ]:
sad_triggered_trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
o = figure(plot_width=1200)
O1 = o.line(xrange, trace.wave, line_color='black')
O2 = o.line(xrange, sad_triggered_trace.wave, line_color='blue')
O3 = o.line(xrange, abs(trace.wave - sad_triggered_trace.wave), line_color='red')

In [ ]:
show(o)

The two waveforms should line up perfectly (there can be large differences around the UART activity -- that's normal).

If `scope.SAD.threshold` is set too aggressively, captures may sometimes pass and sometimes fail.

The SAD trigger module supports issuing multiple triggers: if `scope.SAD.multiple_triggers` is set, then once armed, a trigger will be issued whenever the SAD threshold is met.

If `scope.SAD.multiple_triggers` is not set, then no triggers are issued after the first one, until the scope is re-armed.

`multiple_triggers` can be useful when the power trace for a target operation has some periodic component of interest.

This can be combined with segmented capture (see the [Introduction to Husky notebook](<01 - Introduction to ChipWhisperer-Husky.ipynb>)) to quickly capture samples of interest, by capturing the next trace segment at every SAD trigger event.

On the AES power trace here we can see periods of higher activity that suggest the 10 AES rounds (actually 11, likely due to the initial round key addition).

So we set `scope.adc.segments` to 11 and expect to see the resulting captured segments to appear very similar.

For the SAM4S target we set it up a little different, based on what the power trace looks like.

**If you're using a different target, you will need to find SAD parameters suitable to your target.**

In [ ]:
scope.SAD.multiple_triggers = True

if PLATFORM == 'CW308_STM32F3':
    scope.SAD.reference = trace.wave[13525:]
    scope.SAD.threshold = 200
    scope.adc.segments = 11
    
elif PLATFORM == 'CW308_SAM4S':
    scope.SAD.reference = trace.wave[14940:]
    scope.SAD.threshold = 500
    scope.adc.segments = 10

else:
    raise ValueError('Find appropriate settings for your target.')
    #scope.SAD.reference = trace.wave[?:]
    #scope.SAD.threshold = ?
    #scope.adc.segments = ?

scope.adc.presamples = scope.SAD.sad_reference_length + scope.SAD.latency
scope.adc.samples = 900 # if this is too high, a segmenting error will occur (meaning that segment i needs to be captured before segment i-1 has finished being captured)
scope.adc.segment_cycle_counter_en = False

In [ ]:
sad_triggered_trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))
assert scope.SAD.num_triggers_seen == scope.adc.segments, 'Got %d SAD triggers (expected %d)' % (scope.SAD.num_triggers_seen,  scope.adc.segments)

If the capture fails, what does `scope.SAD.num_triggers_seen` show?

If it's 0, it means the SAD module didn't trigger at all; either your starting reference point is bad, or your threshold is set too low.

If it's greater than 0 but less than `scope.adc.segments`, then the SAD module triggered but not as many times as expected. Again this could be a problem with the reference, the threshold being too low, or maybe the number of segments being too high (can you visually see that many segments on the waveform?).

If it's greater than `scope.adc.segments`, your threshold may be too high (or your starting reference point is bad).

If the capture is not successful, a "segmenting error" may also pop up and provide a clue as to what needs to be fixed. A segmenting error means that the condition for capturing the next segment is true while the previous segment is still being captured. This can happen, for example, if `scope.adc.samples` is too large (e.g. if the SAD trigger fires every 1000 samples, but you're trying to capture 2000 samples per segment). It can also happen if your SAD threshold is too high (e.g. the SAD trigger is firing more often than we intend it to).

(hot tip: you can access the online documentation for any function by running `<function>?` in Jupyter; so `scope.adc.errors?` will show you the documentation for all the `scope.adc` error types!)

Another new Husky feature that can come in handy whenever using multiple triggers is the trigger timestamping feature: Husky records the time delta, in number of ADC clock cyles, between successive triggers.

Let's retrieve these for our previous capture:

In [ ]:
ttimes = scope.trigger.get_trigger_times()
print(ttimes)

You should find that the SAD triggered at constant intervals, as you would expect if it's firing at the same point in each AES round, except:
- for the SAM4S: the last round;
- for the STM32: the first and last rounds.

Which, if you're familiar with AES, sounds perfectly reasonable.

`scope.trigger.get_trigger_times()` isn't just for SAD triggering: you can use it whenever there are multiple triggers in a single capture. In the [sca205](https://github.com/newaetech/chipwhisperer-jupyter/tree/master/courses/sca205) series of ECC attack notebooks, we rely on `get_trigger_times()` to tune the capture trigger parameters, and also to throw out captures that appear to be badly triggered (so that they don't pollute the trace set).

By overlaying all the captured segments, we find that they do line up very well (up to sample 760 or so in the case of the STM32 target):

In [ ]:
from bokeh.palettes import inferno
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import itertools
output_notebook(INLINE)
B = figure(plot_width=1800)
numtrigs = scope.adc.segments
colors = itertools.cycle(inferno(numtrigs))
xrange = range(scope.adc.samples)

max_diff = []
for i in range(scope.adc.samples):
    max_diff.append(max(abs(sad_triggered_trace.wave[i::scope.adc.samples]) - min(abs(sad_triggered_trace.wave[i::scope.adc.samples]))))

for i in range(numtrigs):
    B.line(xrange, sad_triggered_trace.wave[i*scope.adc.samples:(i+1)*scope.adc.samples], color=next(colors))
B.line(xrange, max_diff, color='black', line_width=2)
show(B)

# 4. TraceWhisperer

Triggering from traces is covered in a separate notebook: [TraceWhisperer.ipynb](https://github.com/newaetech/DesignStartTrace/blob/master/jupyter/TraceWhisperer.ipynb).

# 5. Edge Count Triggering

This counts rising and falling edges of an input signal and triggers after a specified number of edges.

This can be very useful when you want to trigger from some protocol that Husky can't decode.

Here we'll demonstrate with UART because it's what we have with our targets, but it can be used for much more than that.

The input signal to the edge counter can be anything that `scope.trigger.triggers` supports (which, in addition to what you could use on CW-Lite/Pro, includes the USERIO data pins).

In [ ]:
scope.default_setup()

In [ ]:
scope.trigger.module = 'edge_counter'
scope.trigger.triggers = 'tio1'
#scope.trigger.triggers = 'tio2'
scope.trigger.edges = 3

We'll use Husky's logic analyzer to visualize the generated trigger. The trigger is a narrow single-cycle pulse of the ADC sampling clock, so we must sample fairly fast, which means we won't see all of the UART traffic.

In [ ]:
scope.trace.enabled = False
scope.LA.enabled = True
scope.LA.clk_source = 'pll'
scope.LA.oversampling_factor = 4
scope.LA.downsample = 1
scope.LA.capture_group = 'CW 20-pin'
scope.LA.trigger_source = 'falling_tio1'
#scope.LA.trigger_source = 'falling_tio2'
scope.LA.capture_depth = 16000
scope.adc.clip_errors_disabled = True

This is so that the internal trigger can be captured by `scope.LA`:

In [ ]:
scope.io.glitch_trig_mcx = 'trigger'

In [ ]:
scope.LA.arm()
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [ ]:
assert not scope.LA.fifo_empty()
raw = scope.LA.read_capture_data()
tio1 = scope.LA.extract(raw, 0)
tio2 = scope.LA.extract(raw, 1)
trig = scope.LA.extract(raw, 7)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import numpy as np
output_notebook(INLINE)

o = figure(plot_width=1800)

xrange = range(len(tio1))
O1 = o.line(xrange, tio1 + 4, line_color='black')
O2 = o.line(xrange, tio2 + 2, line_color='blue')
O3 = o.line(xrange, trig + 0, line_color='red')

legend = Legend(items=[
    LegendItem(label='tio1', renderers=[O1]),
    LegendItem(label='tio2', renderers=[O2]),
    LegendItem(label='internal capture trigger', renderers=[O3]),
])
o.add_layout(legend)

In [ ]:
show(o)

The first falling edge of TIO1 is missed by the `scope.LA` capture (because that's the edge which kicks off the capture).

Accounting for that, you can see that the trigger occurs immediately after the third edge of TIO1.

You can change `scope.trigger.edges` and see the trigger move around accordingly.

If you set it higher than the number of edges that are observed, the capture will time out (because no trigger was generated). If this happens, `scope.trigger.edges_seen` will show how many edges were observed.

In [ ]:
scope.trigger.edges_seen

Try the other trigger sources (don't forget to adjust `scope.LA.trigger_source` accordingly if you want to see the results via `scope.LA` as above).

## 6. Sequenced Triggers

See [04 - Husky Trigger Sequencer.ipynb](04%20-%20Husky%20Trigger%20Sequencer.ipynb) to learn how to sequencer multiple trigger events.